# AGENTS

In [10]:
class Agent:
    """Base class for all agents in the system."""

    def __init__(self, name):
        self.name = name

    def __str__(self):
        return f"{self.name}"

    def describe(self):
        return self.__str__()

# STACK

In [11]:
class Stack:
    def __init__(self):
        self.stack = []

    def is_empty(self):
        return True if len(self.stack) == 0 else False
    
    def push(self, item):
        self.stack.append(item)
        
    def pop(self):
        return None if self.is_empty() else self.stack.pop()
    
    def peek(self):
        return None if self.is_empty () else self.stack[-1]

# QUEUE

In [12]:
class Queue:
    
    def __init__(self):
        self.queue = []
    
    def is_empty (self):
        return True if len(self.queue) == 0 else False
    
    def peek(self):
        return None if self.is_empty () else self.queue[0]
    
    def enqueue (self, item):
        self.queue.append(item)
    
    def dequeue(self):
        return None if self.is_empty() else self.queue.pop(0)
    
    def size(self):
        return len(self.queue)

# AGENT PATIENT

In [13]:
class Patient(Agent):
    """Class representing a patient that interacts with the hospital."""

    def __init__(self, name):
        super().__init__(name)
        self.hospital_stack = Stack()  # Stack of visited hospitals

    def to_dict(self):
        """Returns the patient's information in the same format as hospital.to_dict()."""
        return {
            "patient_name": self.name,
            "hospital_stack": list(self.hospital_stack.stack)  # Assuming direct access
        }

    def enter_hospital(self, hospital_name):
        self.hospital_stack.push(hospital_name)
        print(f'{self.name} entered {hospital_name}.')

    def exit_hospital(self):
        hospital_name = self.hospital_stack.pop()
        if hospital_name:
            print(f'{self.name} exited {hospital_name}.')
        else:
            print(f'{self.name} is not in any hospital.')

    def current_hospital(self):
        return self.hospital_stack.peek()
    
    
    HELP_MESSAGES = {
        "add_patient": "patient add_patient <patient_name>: Add a new patient to the system.",
        "show_all": "patient show_all: Show the list of all patients in the system.",
        "remove_patient": "patient remove_patient <patient_name>: Remove a patient from the system.",
        "request_appointment": "patient request_appointment <patient_name> <hospital_name> <time>: Request an appointment at a specific hospital.",
        "request_urgent_care": "patient request_urgent_care <patient_name> <hospital_name>: Request urgent care at a specific hospital.",
        "enter_hospital": "patient enter_hospital <patient_name> <hospital_name>: Allow a patient to enter the hospital.",
        "exit_hospital": "patient exit_hospital <patient_name> <hospital_name>: Allow a patient to exit the hospital.",
        "request_surgery": "patient request_surgery <patient_name> <hospital_name> <department_name>: Request a surgery in a specific department of the hospital.",
        "check_appointment_status": "patient check_appointment_status <patient_name> <hospital_name>: Check the status of the patient's appointment.",
        "quit": "q: Exit the simulation."
    }

    @classmethod
    def help(cls):
        print("Available commands for patient:")
        for command, description in cls.HELP_MESSAGES.items():
            print(f"- {description}")

# AGENT HOSPITAL

In [14]:
import time

class Hospital(Agent):
    """Class that represents a hospital and its services (departments), doctors, etc."""

    def __init__(self, name):
        super().__init__(name)
        self.departments = []
        self.doctors = []
        self.nurses = []
        self.urgent_care_queue = Queue()
        self.appointments = {}
        self.surgery= {}
        self.rooms = {}

    def to_dict(self):
        return {
            "hospital_name": self.name,
            "departments": self.departments,
            "doctors": self.doctors,
            "nurses": [
                {"nurse_name": nurse["nurse_name"], "department_name": nurse["department_name"]}
                for nurse in self.nurses
            ] if self.nurses else [],
            "appointments": [
                {"patient_name": k, "time": v}
                for k, v in self.appointments.items()
            ],
            "urgent_care_queue": [
                {"patient_name": patient}
                for patient in self.urgent_care_queue.queue
            ],
            "surgery": [
                {"patient_name": k, "hospital": v["hospital"], "department": v["department"]}
                for k, v in self.surgery.items()
            ],
            "rooms": self.rooms
        }

    def add_department(self, department_name):
        self.departments.append(department_name)
        print(f'Department {department_name} added to {self.name}.')

    def assign_nurse(self, nurse_name, department_name):
        self.nurses.append({'nurse_name': nurse_name, 'department_name': department_name})
        print(f'Nurse {nurse_name} assigned to {department_name} in {self.name}.')

    def show_departments(self):
        print(f'Departments in {self.name}:')
        for department in self.departments:
            print(f'- {department}')

    def add_doctor(self, doctor_name):
        if not any(d['doctor_name'] == doctor_name for d in self.doctors):
            self.doctors.append({'doctor_name': doctor_name, 'hospital_name': self.name})
            print(f'Doctor {doctor_name} added to {self.name}.')
        else:
            print(f'Doctor {doctor_name} is already in {self.name}.')

    def assign_doctor(self, doctor_name):
            # Check if doctor is already in this hospital
        if any(doctor["doctor_name"] == doctor_name for doctor in self.doctors):
            print(f"Doctor {doctor_name} is already in {self.name}.")
            return

        # Add doctor with this hospital
        self.doctors.append({"doctor_name": doctor_name, "hospital_name": self.name})
        print(f"Doctor {doctor_name} assigned to {self.name}.")

    def list_doctors(self):
        print(f'Doctors in {self.name}:')
        for doctor in self.doctors:
            print(f"- {doctor['doctor_name']}")

    def show_urgent_queue(self):
        if not self.urgent_care_queue.is_empty():
            print(f"Urgent care queue for hospital '{self.name}':")
            for patient in self.urgent_care_queue.queue:
                print(f"- Patient: {patient['patient_name']}, Timestamp: {round(patient['timestamp'], 0)}")
        else:
            print(f"No patients in urgent care queue for hospital '{self.name}'.")

    def request_urgent_care(self, patient_name):
        timestamp = time.time()
        self.urgent_care_queue.enqueue({'patient_name': patient_name, 'timestamp': timestamp})
        print(f"Urgent care requested for patient {patient_name} in {self.name}.")

    def show_appointments(self):
        if self.appointments:
            print(f"Appointments for hospital {self.name}:")
            for patient, time in self.appointments.items():
                print(f"- Patient: {patient}, Time: {time}")
        else:
            print(f"No appointments scheduled for hospital {self.name}.")

    def request_appointment(self, patient_name, appointment_time):
        self.appointments[patient_name] = appointment_time
        print(f"Appointment scheduled for patient {patient_name} at {appointment_time} in {self.name}.")

    def cancel_appointment(self, patient_name):
        if patient_name in self.appointments:
            del self.appointments[patient_name]
            print(f"Appointment for patient {patient_name} canceled in {self.name}.")
        else:
            print(f"No appointment found for patient {patient_name} in {self.name}.")

    def request_surgery(self, patient_name, department_name):
        self.surgery[patient_name] = {
            "hospital": self.name,
            "department": department_name
        }
        print(f"Surgery requested for patient {patient_name} in {department_name} at {self.name}.")

    def perform_surgery(self, patient_name, department_name):
        if patient_name in self.surgery:
            del self.surgery[patient_name]
        print(f"Surgery performed on patient {patient_name} in {department_name} at {self.name}.")

    def assign_room(self, patient_name, room_name):
        self.rooms[patient_name] = room_name
        print(f"Room {room_name} assigned to patient {patient_name} in {self.name}.")

    HELP_MESSAGES = {
        "add_hospital": "hospital add_hospital <hospital_name>: Add a new hospital to the system.",
        "show_all": "hospital show_all: Show the list of all hospitals in the system.",
        "add_doctor": "hospital add_doctor <hospital_name> <doctor_name>: Add a new doctor to a specific hospital.",
        "assign_doctor": "hospital assign_doctor <hospital_name> <doctor_name>: Assign a doctor to a hospital.",
        "list_doctors": "hospital list_doctors <hospital_name>: List all doctors available in the specified hospital.",
        "show_urgent_queue": "hospital show_urgent_queue <hospital_name>: Show the list of patients in the urgent care queue at a hospital.",
        "show_appointments": "hospital show_appointments <hospital_name>: Show all scheduled appointments for a hospital.",
        "cancel_appointment": "hospital cancel_appointment <patient_name> <hospital_name>: Cancel a patient's appointment at a hospital.",
        "add_department": "hospital add_department <hospital_name> <department_name>: Add a new department in the hospital",
        "assign_nurse": "hospital assign_nurse <hospital_name> <nurse_name> <department_name>: Assign a nurse to a specific department of the hospital",
        "show_departments": "hospital show_departments <hospital_name>: Show the list of departments available in the hospital",
        "perform_surgery": "hospital perform_surgery <hospital_name> <patient_name> <department_name>: Perform a surgery for a patient in a department of the hospital",
        "assign_room": "hospital assign_room <hospital_name> <patient_name> <room_name>: Assign a room to a patient in the hospital",
        "quit": "q: Exit the simulation."
    }

    @classmethod
    def help(cls):
        print("Available commands for hospital:")
        for command, description in cls.HELP_MESSAGES.items():
            print(f"- {description}")

# AGENT MANAGER

In [15]:
import json
import os
class AgentManager:
    """Base class for managing the creation and removal of agents."""

    def __init__(self):
        self.agents = {}  # Dictionary to store agents
        self.TIME_THRESHOLD = 10  # 10-second threshold


    def filter_agents(self, *agent_types):
        return {
            name: agent for name, agent in self.agents.items()
            if isinstance(agent, agent_types)
        }        

    def get_agent_by_name(self, agent_name, agent_type):
       #Returns the agent with the given name and specific type, or None if not found.
        return next((agent for agent in self.agents.values()
                     if isinstance(agent, agent_type) and agent.name == agent_name), None)

    def add_agent(self, agent_type, agent_name): 
        # Adds a new agent to the system, checks if it already exists and adds it to the agents dictionary

        if agent_name in self.agents:
            print(f"Agent {agent_name} already exists.")
            return

        if isinstance(agent_type, type):
            self.agents[agent_name] = agent_type(agent_name)
        else:
            print(f"Invalid agent type: {agent_type}")
            return
       
        print(f'{agent_type.__name__} {agent_name} added to the system.')


    def remove_agent(self, agent_name):
        #Removes an agent from the system.
        if agent_name in self.agents:
            del self.agents[agent_name]
            print(f'Agent {agent_name} removed from the system.')
        else:
            print(f'Agent {agent_name} not found.')

    def list_agents(self, agent_type=None):
        #Displays all agents or filters by patients or hospitals in the system.
        if agent_type:
            print(f"Current {agent_type.__name__}(s):")
            filtered_agents = self.filter_agents(agent_type)
            for agent in filtered_agents.values():
                print(agent.describe())
        else:
            print("Current agents:")
            for agent in self.agents.values():
                print(agent.describe())
    
    def check_appointment_status(self, patient, hospital):  # Move to agent manager if needed
        #Displays the status of the patient's appointment at the specified hospital."""
        if patient.current_hospital() != hospital.name:
            print(f"{patient.name} is not registered in {hospital.name}.")
            return

        if patient.name in hospital.appointments:
            print(f"{patient.name} has an appointment scheduled at {hospital.appointments[patient.name]} in {hospital.name}.")
        else:
            print(f"{patient.name} has no registered appointment in {hospital.name}.")

    
    def validate_patient_location(self, patient, hospital_name):
        # Validate if the patient is in the specified hospital.
        if patient.current_hospital() is None:
            print(f'Patient {patient.name} cannot perform this action because not in it.')
            return False
        elif patient.current_hospital() != hospital_name:
            print(f'Patient {patient.name} is in a different hospital: {patient.current_hospital()}.')
            return False
        return True

    def load_agents_from_file(self, file_path):
        """Load agents from a JSON file."""     

        AGENT_CLASSES = {
            "hospitals": Hospital,
            "patients": Patient,            
        }

        try:
            with open(file_path, 'r') as file:
                data = json.load(file)

            for group_name, agent_class in AGENT_CLASSES.items():
                for agent_name, agent_data in data.get(group_name, {}).items():
                    self.add_agent(agent_class, agent_name)

                   # Retrieve the instance we just created
                    agent = self.get_agent_by_name(agent_name, agent_class)

                   # Specific logic for certain types
                    if isinstance(agent, Hospital):
                        for d in agent_data.get("departments", []):
                            agent.add_department(d)
                        for doctor in agent_data.get("doctors", []):
                            agent.add_doctor(doctor)
                        for nurse in agent_data.get("nurses", []):
                            agent.assign_nurse(nurse["nurse_name"], nurse["department_name"])
                        for appt in agent_data.get("appointments", []):
                            agent.request_appointment(appt["patient_name"], appt["time"])
                        for p in agent_data.get("urgent_care_queue", []):
                            agent.request_urgent_care(p["patient_name"])
                        for p_name, r_name in agent_data.get("rooms", {}).items():
                            agent.assign_room(p_name, r_name)

                    elif isinstance(agent, Patient):
                        if agent_data.get("current_hospital"):
                            agent.enter_hospital(agent_data["current_hospital"])

            print(f"--- Agents loaded" )
            print(f"successfully from {file_path}. ---")

        except FileNotFoundError:
            print(f"Error: File '{file_path}' not found.")
        except json.JSONDecodeError:
            print(f"Error: File '{file_path}' is not a valid JSON file.")
        except Exception as e:
            print(f"An error occurred while loading agents: {e}")

    def save_agents_to_file(self, file_path):
    
        data = {}

        for name, agent in self.agents.items():
            agent_type = type(agent).__name__.lower() + "s"  # example: "Hospital" → "hospitals"
            if agent_type not in data:
                data[agent_type] = {}
            data[agent_type][name] = agent.to_dict()

        with open(file_path, "w") as file:
            json.dump(data, file, indent=4)      
        print(f"Archive saved in: {os.path.abspath(file_path)}")

### GLOBAL DICTIONARY AGENTS

In [16]:
agents = {}# Global dictionary to store agents

# CITY SIMULATION

In [17]:
class CitySimulation:
    """Main class to manage the hospital system simulation."""

    def __init__(self):
        self.agent_manager = AgentManager()
        self.ERROR_MESSAGES = {
        "invalid_command": "Error: Unknown command. Enter ? to see available commands.",
        "hospital_not_found": "Error: Hospital '{name}' not found.",
        "patient_not_found": "Error: Patient '{name}' not found.",
        "invalid_format": "Error: Invalid format. Use: '{expected_format}'"
    }


    def command_loop(self):
        print("Starting simulation... type 'q' to quit")
        while True:
            command = input('> ').strip()
            if command == 'q':
                break
            self.process_command(command)

    def process_command(self, command):
        parts = command.split()
        if not parts:
            return
        match parts[0]:
            case '?': self.process_help(parts)
            case 'load_agents': self.process_load_agents(parts)
            case 'save_agents': self.process_save_agents(parts)
            case 'hospital': self.process_hospital_command(parts)
            case 'patient': self.process_patient_command(parts)
            case _: print(self.ERROR_MESSAGES["invalid_command"])

    def process_help(self, parts):
        if len(parts) == 2:
            match parts[1]:
                case 'hospital': Hospital.help()
                case 'patient': Patient.help()
                case _: print("Unknown help command. Use '? hospital' or '? patient'.")
        else:
            print("""
            Available commands:
            - ? hospital: Show available commands for hospitals.
            - ? patient: Show available commands for patients.
            - load_agents <path>: Load agents from a JSON file.
            - save_agents <path>: Save agents to a JSON file.
            - q: Exit the simulation.
            """)


    def process_load_agents(self, parts):
        if len(parts) != 2:
            print(self.ERROR_MESSAGES["invalid_format"].format(expected_format="load_agents <path>"))
            return
        _, file_path = parts
        self.agent_manager.load_agents_from_file(file_path)

    def process_save_agents(self, parts):
        if len(parts) != 2:
            print(self.ERROR_MESSAGES["invalid_format"].format(expected_format="save_agents <path>"))
            return
        _, file_path = parts
        self.agent_manager.save_agents_to_file(file_path)

    def process_hospital_command(self, parts):
        if len(parts) < 2:
            print(self.ERROR_MESSAGES["invalid_format"].format(expected_format="hospital <comando> [...]"))
            return
        match parts[1]:
            case 'add_hospital':
                if self.validate_command(parts, 3, "invalid_format", "hospital add_hospital <nombre>"):
                    _, _, hospital_name = parts
                    self.agent_manager.add_agent(Hospital, hospital_name)
            case 'show_all':
                if self.validate_command(parts, 2, "invalid_format", "hospital show_all"):
                    self.agent_manager.list_agents(Hospital)
            case 'add_department':
                if self.validate_command(parts, 4, "invalid_format", "hospital add_department <hospital> <departamento>"):
                    _, _, hospital_name, department = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        hospital.add_department(department)
            case 'assign_nurse':
                if self.validate_command(parts, 5, "invalid_format", "hospital assign_nurse <hospital> <enfermera> <departamento>"):
                    _, _, hospital_name, nurse_name, department = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        hospital.assign_nurse(nurse_name, department)
            case 'add_doctor':
                if self.validate_command(parts, 4, "invalid_format", "hospital add_doctor <hospital> <doctor>"):
                    _, _, hospital_name, doctor_name = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        hospital.add_doctor(doctor_name)
            case 'assign_doctor':
                if self.validate_command(parts, 4, "invalid_format", "hospital assign_doctor <hospital> <doctor>"):
                    _, _, hospital_name, doctor_name = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        hospital.assign_doctor(doctor_name)
            case 'list_doctors':
                if self.validate_command(parts, 3, "invalid_format", "hospital list_doctors <hospital>"):
                    _, _, hospital_name = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        hospital.list_doctors()
            case 'show_urgent_queue':
                if self.validate_command(parts, 3, "invalid_format", "hospital show_urgent_queue <hospital>"):
                    _, _, hospital_name = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        hospital.show_urgent_queue()
            case 'show_appointments':
                if self.validate_command(parts, 3, "invalid_format", "hospital show_appointments <hospital>"):
                    _, _, hospital_name = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        hospital.show_appointments()
            case 'cancel_appointment':
                if self.validate_command(parts, 4, "invalid_format", "hospital cancel_appointment <patient> <hospital>"):
                    _, _,  patient_name, hospital_name  = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        hospital.cancel_appointment(patient_name)
            case 'show_departments':
                if self.validate_command(parts, 3, "invalid_format", "hospital show_departments <hospital>"):
                    _, _, hospital_name = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        hospital.show_departments()
            case 'perform_surgery':
                if self.validate_command(parts, 5, "invalid_format", "hospital perform_surgery <hospital> <patient> <departamento>"):
                    _, _, hospital_name, patient_name, department = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        hospital.perform_surgery(patient_name, department)
            case 'assign_room':
                if self.validate_command(parts, 5, "invalid_format", "hospital assign_room <hospital> <patient> <room>"):
                    _, _, hospital_name, patient_name, room_name = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        hospital.assign_room(patient_name, room_name)
            case 'remove_hospital':
                if self.validate_command(parts, 3, "invalid_format", "hospital remove_hospital <nombre>"):
                    _, _, hospital_name = parts
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if hospital:
                        # Group the resources that must be empty before removal
                        hospital_resources = [                                                                                 
                            ("appointments", hospital.appointments),
                            ("surgery", hospital.surgery),
                        ]
                        for agent in self.agent_manager.agents.values():
                                    if isinstance(agent, Patient) and hospital_name in agent.hospital_stack.stack:
                                        hospital_resources.append(("patients", agent.name))
                                        break

                        # Verify if any of the resources are occupied
                        occupied = [name for name, resource in hospital_resources if resource]

                        if occupied:
                            print(f"Cannot remove hospital {hospital_name} because it has active: {', '.join(occupied)}.")
                        else:
                            self.agent_manager.remove_agent(hospital_name)


            case _:
                print(self.ERROR_MESSAGES["invalid_command"])

    def process_patient_command(self, parts):
        if len(parts) < 2:
            print(self.ERROR_MESSAGES["invalid_format"].format(expected_format="patient <comando> [...]"))
            return
        match parts[1]:
            case 'add_patient':
                if self.validate_command(parts, 3, "invalid_format", "patient add_patient <nombre>"):
                    _, _, patient_name = parts
                    self.agent_manager.add_agent(Patient, patient_name)
            case 'remove_patient':
                if self.validate_command(parts, 3, "invalid_format", "patient remove_patient <nombre>"):
                    _, _, patient_name = parts
                    self.agent_manager.remove_agent(patient_name)
            case 'show_all':
                if self.validate_command(parts, 2, "invalid_format", "patient show_all"):
                    self.agent_manager.list_agents(Patient)
            case 'enter_hospital':
                if self.validate_command(parts, 4, "invalid_format", "patient enter_hospital <patient> <hospital>"):
                    _, _, patient_name, hospital_name = parts
                    patient = self.get_agent_or_error(patient_name, Patient, "patient_not_found")
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if patient and hospital:
                        patient.enter_hospital(hospital_name)
            case 'exit_hospital':
                if self.validate_command(parts, 4, "invalid_format", "patient exit_hospital <patient> <hospital>"):
                    _, _, patient_name, hospital_name = parts
                    patient = self.get_agent_or_error(patient_name, Patient, "patient_not_found")
                    if patient and self.agent_manager.validate_patient_location(patient, hospital_name):
                        patient.exit_hospital()
            case 'request_appointment':
                if self.validate_command(parts, 5, "invalid_format", "patient request_appointment <patient> <hospital> <hora>"):
                    _, _, patient_name, hospital_name, time = parts
                    patient = self.get_agent_or_error(patient_name, Patient, "patient_not_found")
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if patient and hospital and self.agent_manager.validate_patient_location(patient, hospital_name):
                        hospital.request_appointment(patient_name, time)
            case 'request_urgent_care':
                if self.validate_command(parts, 4, "invalid_format", "patient request_urgent_care <patient> <hospital>"):
                    _, _, patient_name, hospital_name = parts
                    patient = self.get_agent_or_error(patient_name, Patient, "patient_not_found")
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if patient and hospital and self.agent_manager.validate_patient_location(patient, hospital_name):
                        hospital.request_urgent_care(patient_name)
            case 'check_appointment_status':
                if self.validate_command(parts, 4, "invalid_format", "patient check_appointment_status <patient> <hospital>"):
                    _, _, patient_name, hospital_name = parts
                    patient = self.get_agent_or_error(patient_name, Patient, "patient_not_found")
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if patient and hospital:
                        self.agent_manager.check_appointment_status(patient, hospital)

            case 'request_surgery':
                if self.validate_command(parts, 5, "invalid_format", "patient request_surgery <patient> <hospital> <departamento>"):
                    _, _, patient_name, hospital_name, department = parts
                    patient = self.get_agent_or_error(patient_name, Patient, "patient_not_found")
                    hospital = self.get_agent_or_error(hospital_name, Hospital, "hospital_not_found")
                    if patient and hospital and self.agent_manager.validate_patient_location(patient, hospital_name):
                        hospital.request_surgery(patient_name, department)
            case _:
                print(self.ERROR_MESSAGES["invalid_command"])

    def get_agent_or_error(self, agent_name, agent_type, error_key):
        agent = self.agent_manager.get_agent_by_name(agent_name, agent_type)
        if not agent:
            print(self.ERROR_MESSAGES[error_key].format(name=agent_name))
        return agent

    def validate_command(self, parts, expected_length, error_key, expected_format):
        if len(parts) != expected_length:
            print(self.ERROR_MESSAGES[error_key].format(expected_format=expected_format))
            return False
        return True

In [ ]:
if __name__ == "__main__":
    sim = CitySimulation()
    sim.command_loop()

Starting simulation... type 'q' to quit


>  patient add_patient JuanPerez


Patient JuanPerez added to the system.


>  hospital add_hospital ClinicaUniversitaria


Hospital ClinicaUniversitaria added to the system.


...

Error: Unknown command. Enter ? to see available commands.


>  patient perform_surgery ClinicaUniversitaria JuanPerez cardio


Error: Unknown command. Enter ? to see available commands.


>  hospital perform_surgery ClinicaUniversitaria JuanPerez cardio


Surgery performed on patient JuanPerez in cardio at ClinicaUniversitaria.
